<a href="https://colab.research.google.com/github/shagufta24/Personalized_Writing_Assistant/blob/main/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `my_token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when 

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig

BASE_MODEL_ID = "mistralai/Mistral-7B-v0.3"
drive_path = "/content/drive/MyDrive/cs598pen/mistral-finetune-reddit"

In [ ]:
base_model_path = drive_path + "/base-model"
lora_adapter_path = drive_path + "/checkpoint-125"

In [ ]:
# Enable logging
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained (
    base_model_path,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [ ]:
test_tokenizer = AutoTokenizer.from_pretrained(base_model_path, add_bos_token=True, trust_remote_code=True)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
# Reload the fine-tuned model with LoRA adapter
fine_tuned_model = PeftModel.from_pretrained(base_model, lora_adapter_path)

In [ ]:
!pip install rouge_score
!pip install mauve
import json
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import mauve

In [ ]:
# File paths
TEST_FILE = "/content/testing_data_reddit.jsonl"

# Load test data
def load_test_data(file_path):
    with open(file_path, "r") as f:
        return [json.loads(line.strip()) for line in f]

# Load test dataset
test_data = load_test_data(TEST_FILE)

In [ ]:
# Helper to generate responses
def generate_response(model, tokenizer, prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prompt template
def create_prompt(comment, user_name="1percentchick"):
    return f"The following is a Reddit comment by {user_name}: {comment}"

In [ ]:
# Evaluate Metrics
def evaluate_metrics(test_data, model, tokenizer, user_name="1percentchick"):
    bleu_scores = []
    rouge_scores = []
    references = []
    predictions = []

    rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    for example in test_data:
        prompt = create_prompt(example["comment"], user_name)
        generated_response = generate_response(model, tokenizer, prompt)

        # Reference text
        reference = example["comment"]
        references.append(reference)
        predictions.append(generated_response)

        # BLEU score
        bleu = sentence_bleu([reference.split()], generated_response.split())
        bleu_scores.append(bleu)

        # ROUGE scores
        rouge_result = rouge.score(reference, generated_response)
        rouge_scores.append(rouge_result)

    return bleu_scores, rouge_scores, references, predictions

In [ ]:
# Compute MAUVE
def compute_mauve(references, predictions):
    mauve_result = mauve.compute_mauve(
        p_text=predictions,
        q_text=references,
        device_id=0  # Set to -1 for CPU
    )
    return mauve_result.mauve

In [ ]:
# Evaluate Base Model
print("Evaluating Base Model...")
base_bleu, base_rouge, base_refs, base_preds = evaluate_metrics(test_data, base_model, tokenizer)

# Evaluate Fine-Tuned Model
print("Evaluating Fine-Tuned Model...")
fine_tuned_bleu, fine_tuned_rouge, fine_refs, fine_preds = evaluate_metrics(test_data, fine_tuned_model, tokenizer)

# Compute MAUVE
print("Computing MAUVE...")
base_mauve = compute_mauve(base_refs, base_preds)
fine_tuned_mauve = compute_mauve(fine_refs, fine_preds)

In [ ]:
# Display Results
print("\nBase Model Metrics:")
print(f"Average BLEU: {sum(base_bleu) / len(base_bleu):.4f}")
print(f"Average ROUGE: {sum([x['rougeL'].fmeasure for x in base_rouge]) / len(base_rouge):.4f}")
print(f"MAUVE: {base_mauve:.4f}")

print("\nFine-Tuned Model Metrics:")
print(f"Average BLEU: {sum(fine_tuned_bleu) / len(fine_tuned_bleu):.4f}")
print(f"Average ROUGE: {sum([x['rougeL'].fmeasure for x in fine_tuned_rouge]) / len(fine_tuned_rouge):.4f}")
print(f"MAUVE: {fine_tuned_mauve:.4f}")